# Notebook 1 : Découverte de Pandas

In [ ]:
import pandas as pd

## Iris

Nous considérons le jeu de données connu sous le nom des [Iris de Fisher](https://fr.wikipedia.org/wiki/Iris_de_Fisher). Répondez aux questions suivantes à l'aide du module Pandas.

1. Charger le jeu de données dans un dataframe `iris` à partir du fichier `iris.csv` et afficher le dataframe.

In [ ]:
iris = pd.read_csv("data/iris.csv")
iris

2. Sélectionner les variables `PetalWidth` et `Species`.

In [ ]:
# Sélection par nom
iris[["PetalWidth", "Species"]]

# Sélection avec filter
iris.filter(items=["PetalWidth", "Species"])

3. Extraire le sous-dataframe qui contient uniquement les iris des espèces `versicolor` ou `virginica`.

In [ ]:
# Avec l'opérateur "|"
iris[(iris.Species == "versicolor") | (iris.Species == "virginica")]

# Avec la méthode isin
iris[iris.Species.isin(["versicolor", "virginica"])]

4. Compter le nombre d'iris de l'espèce `setosa`.

In [ ]:
# Avec la fonction len
len(iris[iris.Species == "setosa"])

# Avec l'agrégateur count
iris[iris.Species == "setosa"].Species.count()

5. Calculer la moyenne de la variable `PetalWidth` pour les iris de l'espèce `versicolor`.

In [ ]:
# Avec la méthode mean
iris[iris.Species == "versicolor"].PetalWidth.mean()

# Avec l'agrégateur mean
iris[iris.Species == "versicolor"].PetalWidth.agg("mean")

6. Ajouter une variable `SumWidth` qui contient la somme des variables `PetalWidth` et `SepalWidth`.

In [ ]:
iris["SumWidth"] = iris.PetalWidth + iris.SepalWidth
iris

7. Calculer la moyenne et la variance de la variable `SepalLength` pour chaque espèce.

In [ ]:
iris.groupby("Species").SepalLength.agg(["mean", "var"])

8. Calculer la moyenne de toutes les variables pour chaque espèce. Afficher le résultat au format large et au format long.

In [ ]:
# Moyennes avec la méthode mean
format_large = iris.groupby("Species").mean()
format_large

In [ ]:
# Passage au format long
(
    format_large.reset_index()
    .melt(
        id_vars=["Species"],
        var_name="Variable",
        value_name="Mean",
    )
)

## Houston flights

Nous considérons le jeux de données `hflights` relatif à des vols au départ des aéroports *Houston George Bush Intercontinental Airport* (IAH) et *William P. Hobby Airport* (HOU).

1. Charger le jeu de données dans un dataframe `hflights` à partir du fichier `hflights.csv`

In [ ]:
hflights = pd.read_csv("data/hflights.csv")

2. Sélectionner les variable `Dest`, `Distance`, `TaxiIn` et `TaxiOut` avec la méthode `filter`.

In [ ]:
# Sélection avec items
hflights.filter(items=["Dest", "Distance", "TaxiIn", "TaxiOut"])

# Sélection par expression régulière
hflights.filter(regex="^(D.st|Taxi)")

3. Sélectionner les variables `DepTime`, `ArrTime`, `ActualElapsedTime` et `AirTime` avec la méthode `filter` et le paramètre `like`.

In [ ]:
hflights.filter(like="Time")

4. Ajouter une variable `ActualGroundTime` qui correspond à `ActualElapsedTime` moins `AirTime`.

In [ ]:
hflights["ActualGroundTime"] = hflights.ActualElapsedTime - hflights.AirTime

5. Ajouter une variable `AverageSpeed` qui donne la vitesse moyenne du vol et ordonner la table selon les valeurs décroissantes de cette variable.

In [ ]:
hflights["AverageSpeed"] = hflights.Distance / hflights.AirTime
hflights.sort_values(by="AverageSpeed", ascending=False, inplace=True)
hflights

6. Sélectionner les vols à destination de `JFK`.

In [ ]:
hflights[hflights.Dest == "JFK"]

7. Compter le nombre de vols à destination de `JFK`.

In [ ]:
len(hflights[hflights.Dest == "JFK"])

8. Agréger les données `hflights` pour obtenir :
- `n` : le nombre total de vols,
- `n_dest`: le nombre total de destinations distinctes,
- `n_carrier` : le nombre total de compagnies distinctes.

In [ ]:
# Avec agg
hflights.agg(
    n=pd.NamedAgg(column="FlightNum", aggfunc="count"),
    n_dest=pd.NamedAgg(column="Dest", aggfunc=pd.Series.nunique),
    n_carrier=pd.NamedAgg(column="UniqueCarrier", aggfunc=pd.Series.nunique),
)

# Avec un nouveau dataframe
pd.DataFrame(
    {
        "n": len(hflights),
        "n_dest": hflights.Dest.nunique(),
        "n_carrier": hflights.UniqueCarrier.nunique(),
    },
    index=[0] # Index obligatoire si on ne passe pas des listes
)

9. Agréger les données des vols de la compagnie `AA` pour obtenir :
- le nombre total de vols,
- le nombre total de vols annulés,
- la valeur moyenne de `ArrDelay`.

In [ ]:
aa = hflights[hflights.UniqueCarrier == "AA"]
pd.DataFrame(
    # Index non nécessaire si on n'utilise des listes
    {
        "n": [len(aa)],
        "n_cancel": [aa.Cancelled.sum()],
        "mean": [aa.ArrDelay.mean()],
    },
)

10. Calculer pour chaque compagnie :
- le nombre total de vols,
- la valeur moyenne de `AirTime`.

In [ ]:
hflights.groupby("UniqueCarrier").agg(
    n=pd.NamedAgg(column="FlightNum", aggfunc="count"),
    mean=pd.NamedAgg(column="AirTime", aggfunc="mean"),
)

11. Ordonner les compagnies en fonction des retards moyens au départ.

In [ ]:
hflights.groupby("UniqueCarrier").agg(
    mean=pd.NamedAgg(column="DepDelay", aggfunc="mean"),
).sort_values(by="mean")